## Drawing polycyclic Astral chiral point-line 3-configurations.


We represent a point-line incidence structure as a dictionary of points
p: (x,y)
and a dictionary of line segments:
L : (p,q)
It is important that the line segments span the points at gratest distance.


<img src="astral_5_2_2_1.png" width=600 height = 300>

In this notebook we will focus on astral 3-configurations. We will denote them
as $\cal A(m,j,k,t)$. The configuration $\cal A(5,2,2,1)$ is shown. It is saved in the file
astral_5_2_2_1.pdf. (It as been manually translated into astral_5_2_2_1.png. In general, there are two configurations for each set of parameters. The other one is stored as astral_5_2_2_1'.pdf.

### version of August 13, 2020
### by Tomaž Pisanski

In [ ]:
from functools import reduce
import numpy as np
from itertools import permutations
from sympy import primefactors
#print(primefactors(600851475143)[-1])



def det(a, b):
    return a[0] * b[1] - a[1] * b[0]

def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1]) #Typo was here

    div = det(xdiff, ydiff)
    if div == 0:
       raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y

def draw_cfg(vts,lns,epsilon=0.1,r=3,cline="black",cpoint="red",th=0.2):
    """Draw configuration."""
    plt = Graphics()
    for k in lns:
        (p,q) = lns[k]
        (xp,yp) = vts[p]
        (xq,yq) = vts[q]
        xxp = xp + epsilon*(xp-xq)
        xxq = xq + epsilon*(xq-xp)
        yyp = yp + epsilon*(yp-yq)
        yyq = yq + epsilon*(yq-yp)
        plt += line(((xxp,yyp),(xxq,yyq)),color = cline,thickness=th)
    for m in vts:
        (x,y) = vts[m]
        plt += circle((x,y),r,color = cline)
        plt += circle((x,y),r-1,color = cpoint,fill=True)
    return plt

def cyclic_cover(n,volts,base=None):
    """Each base vertex is given by a radius and offset angle. Each voltage is given by triple, (from,to,i)"""
    if base == None:
        voltgr = DiGraph(volts,multiedges=True)
        basevts = voltgr.vertices()
        R = 10
        k = len(basevts)
        base=[(R*(i+1),0) for i in range(k)]
    verts = {}
    for j in range(len(base)):
        R = base[j][0]
        a = base[j][1]*pi/180
        for i in range(n):
            verts[n*j+i] = (R*cos(a+2*pi*i/n), R*sin(a+2*pi*i/n))
    eds = []
    for (frm,t,i) in volts:
        for j in range(n):
            eds += [[n*frm+j,n*t+(i+j)%n]]
#    print(eds)
    gr = Graph(eds,multiedges=True,pos=verts)
    return gr

def is_self_dual(gr):
    assert gr.is_connected()
    gru = gr.automorphism_group()
    verts = gr.vertices()
    v = verts[0]
    neigh = gr.neighbors(v)
    (a,b) = gr.bipartite_sets()
    if v in b:
        a,b = b,a
    orbv = set(gru.orbit(v))
    inter = orbv.isdisjoint(b)
    return not inter




def circle_line_segment_intersection(circle_center, circle_radius, pt1, pt2, full_line=True, tangent_tol=1e-9):
    """ Find the points at which a circle intersects a line-segment.  This can happen at 0, 1, or 2 points.

    :param circle_center: The (x, y) location of the circle center
    :param circle_radius: The radius of the circle
    :param pt1: The (x, y) location of the first point of the segment
    :param pt2: The (x, y) location of the second point of the segment
    :param full_line: True to find intersections along full line - not just in the segment.  False will just return intersections within the segment.
    :param tangent_tol: Numerical tolerance at which we decide the intersections are close enough to consider it a tangent
    :return Sequence[Tuple[float, float]]: A list of length 0, 1, or 2, where each element is a point at which the circle intercepts a line segment.

    Note: We follow: http://mathworld.wolfram.com/Circle-LineIntersection.html
    (from StackOverflow)
    """

    (p1x, p1y), (p2x, p2y), (cx, cy) = pt1, pt2, circle_center
    (x1, y1), (x2, y2) = (p1x - cx, p1y - cy), (p2x - cx, p2y - cy)
    dx, dy = (x2 - x1), (y2 - y1)
    dr = (dx ** 2 + dy ** 2)**.5
    big_d = x1 * y2 - x2 * y1
    discriminant = circle_radius ** 2 * dr ** 2 - big_d ** 2

    if discriminant < 0:  # No intersection between circle and line
        return []
    else:  # There may be 0, 1, or 2 intersections with the segment
        intersections = [
            (cx + (big_d * dy + sign * (-1 if dy < 0 else 1) * dx * discriminant**.5) / dr ** 2,
             cy + (-big_d * dx + sign * abs(dy) * discriminant**.5) / dr ** 2)
            for sign in ((1, -1) if dy < 0 else (-1, 1))]  # This makes sure the order along the segment is correct
        if not full_line:  # If only considering the segment, filter out intersections that do not fall within the segment
            fraction_along_segment = [(xi - p1x) / dx if abs(dx) > abs(dy) else (yi - p1y) / dy for xi, yi in intersections]
            intersections = [pt for pt, frac in zip(intersections, fraction_along_segment) if 0 <= frac <= 1]
        if len(intersections) == 2 and abs(discriminant) <= tangent_tol:  # If line is tangent to circle, return just one point (as both intersections have same location)
            return [intersections[0]]
        else:
            return intersections

def define_circle(p1, p2, p3):
    """
    Returns the center and radius of the circle passing the given 3 points.
    In case the 3 points form a line, returns (None, infinity).
    """
    temp = p2[0] * p2[0] + p2[1] * p2[1]
    bc = (p1[0] * p1[0] + p1[1] * p1[1] - temp) / 2
    cd = (temp - p3[0] * p3[0] - p3[1] * p3[1]) / 2
    det = (p1[0] - p2[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p2[1])

    if abs(det) < 1.0e-6:
        return (None, np.inf)

    # Center of circle
    cx = (bc*(p2[1] - p3[1]) - cd*(p1[1] - p2[1])) / det
    cy = ((p1[0] - p2[0]) * cd - (p2[0] - p3[0]) * bc) / det

    radius = np.sqrt((cx - p1[0])**2 + (cy - p1[1])**2)
    return ((cx, cy), radius)

def dist(pta,ptb):
    xa = pta[0]
    ya = pta[1]
    xb = ptb[0]
    yb = ptb[1]
    res = sqrt(float((xa-xb)*(xa-xb) + (ya-yb)*(ya-yb)))
    return res

def draw_astral_3_cfg(m,j,k,t,R = 100,f=0,circ=True,r=2,fprefix=None):
    """Implementation of the algorithm from 
    'Chiral Astral Realizations of Cyclic 3-configurations'
    by Berman, DeOrsey, Faudree, Pisanski and Žitnik,
    Dicrete and Computational Geometry (2021)"""

    vv = [(float(R*cos(2*i*pi/m)), float(R*sin(2*i*pi/m))) for i in range(m)]
    ((cx,cy),radius) = define_circle((0,0),vv[t],vv[(t-k)%m])
    inter = circle_line_segment_intersection((float(cx),float(cy)), radius, vv[0], vv[j], full_line=True, tangent_tol=1e-9)
    inters = list(inter)
    w0x = inters[f][0]
    w0y = inters[f][1]
    Rw = float(sqrt(w0x*w0x + w0y*w0y))
    phi = arctan2(w0y,w0x)
    ww = [(float(Rw*cos(phi + 2*i*pi/m)), float(Rw*sin(phi + 2*i*pi/m))) for i in range(m)]
    Rmax = R
    if Rw > R:
        Rmax = Rw
        fact = float(R)/float(Rw)
        R = fact*R
        Rw = fact*Rw
        vv = [(float(R*cos(2*i*pi/m)), float(R*sin(2*i*pi/m))) for i in range(m)]
        ww = [(float(Rw*cos(phi + 2*i*pi/m)), float(Rw*sin(phi + 2*i*pi/m))) for i in range(m)]
        ((cx,cy),radius) = define_circle((0,0),vv[t],vv[(t-k)%m])        
    pts = {i:vv[i] for i in range(m)}
    for i in range(m):
        pts[i+m] = ww[i]
    ia = 0
    ib = j
    ic = m
    dab = dist(pts[ia],pts[ib])
    dac = dist(pts[ia],pts[ic])
    dbc = dist(pts[ib],pts[ic])
    dmax = dab
    if dac > dab:
        dmax = dac
        ib,ic = ic,ib
    if dbc > dmax: # max disntance is between ia and ib
        ia = ic
    iia = ia%m
    kia = ia//m
    iib = ib%m
    kib = ib//m
    lns1 = [((iia + i)%m + kia*m,(iib + i)%m +kib*m) for i in range(m)]
    ia = m
    ib = m + k
    ic = t
    dab = dist(pts[ia],pts[ib])
    dac = dist(pts[ia],pts[ic])
    dbc = dist(pts[ib],pts[ic])
    dmax = dab
    if dac > dab:
        dmax = dac
        ib,ic = ic,ib
    if dbc > dmax: # max disntance is between ia and ib
        ia = ic
    iia = ia%m
    kia = ia//m
    iib = ib%m
    kib = ib//m
    lns2 = [((iia + i)%m + kia*m,(iib + i)%m +kib*m) for i in range(m)]
    lnsall = lns1+lns2
    lns = {ii:lnsall[ii] for ii in range(len(lnsall))}
    plt1 = draw_cfg(pts,lns,r=r)
    if circ:
        plt = plt1 + circle((cx,cy),radius)
    else:
        plt = plt1
    prime = ""
    if f == 1:
        prime = "'"
    if not fprefix == None:
        fname=fprefix+str(m)+"_"+str(j)+"_"+str(k)+"_"+str(t)+prime+".pdf"
        plt.save(fname,axes=False)
    return plt

def intersection_points_astral_3_cfg(m,j,k,t,R = 100,f=0):
    """Implementation of the algorithm from 
    'Chiral Astral Realizations of Cyclic 3-configurations'
    by Berman, DeOrsey, Faudree, Pisanski and Žitnik,
    that determines the intersection points of circle and line.
    Discrete and Computational Geometry (2021)"""

    vv = [(float(R*cos(2*i*pi/m)), float(R*sin(2*i*pi/m))) for i in range(m)]
    ((cx,cy),radius) = define_circle((0,0),vv[t],vv[(t-k)%m])
    inter = circle_line_segment_intersection((float(cx),float(cy)), radius, vv[0], vv[j], full_line=True, tangent_tol=1e-9)
    inters = list(inter)
    return inters
 

def Levi_graph_astral_3(m,j,k,t):
    volts = [(0,1,0),(0,1,j),(0,2,0),(3,2,0),(3,2,k),(3,1,t)]
    gr = cyclic_cover(m,volts)
#    gr.show()
    return gr 

def properties_Levi_graph(gr):
    """check some properties of graph gr"""
    sd = "--"
    if is_self_dual(gr):
        sd = "sd"
    vt = "--"
    if gr.is_vertex_transitive():
        vt = "vt"
    et = "--"
    if gr.is_edge_transitive():
        et = "et"
    at = "--"
    if gr.is_arc_transitive():
        at = "at"
    cn = "--"
    if gr.is_connected():
        cn = "cn"
    grth = gr.girth()
    gru = gr.automorphism_group()
    odrr = gru.order()
    norbs = len(gru.orbits())
    return (sd,vt,et,at,cn,grth,odrr,norbs)

def generate_astral3c(m):
    """Generate astral connected 3-configurations (2m_3)."""
    gphs ={}
    realisable = []
    canon = {}
    for j in range(1,(m+1)//2):
        d1 = gcd(m,j)
        for k in range(1,(m+1)//2):
            d2 = gcd(d1,k)
            for t in range(1,m):
                if j == t:
                    continue
                if k == t:
                    continue
                if (j+k) == t:
                    continue
                d3 = gcd(d2,t)
                if d3 > 1:
                    continue
                g = Levi_graph_astral_3(m,j,k,t)
                inpts = intersection_points_astral_3_cfg(m,j,k,t)
                nreal = len(inpts)
                gr = g.copy(immutable=True)
                (sd,vt,et,at,cn,grth,odrr,norbs) = properties_Levi_graph(gr)
                if nreal > 0:
                    realisable.append((m,j,k,t,sd,vt,et,at,seen,grth,odrr,norbs,nreal))
                seen = "  "
                for gr1 in gphs:
                    if gr1.is_isomorphic(gr):
                        seen = "**"
                        gphs[gr1].append((m,j,k,t,sd,vt,et,at,seen,grth,odrr,norbs,nreal))
                        canon[(m,j,k,t)] = gphs[gr1][0]
                        break
                if seen == "  ":
                    gphs[gr] = [(m,j,k,t,sd,vt,et,at,seen,grth,odrr,norbs)]
                    canon[(m,j,k,t)] = gphs[gr][0]
#                print(m,j,k,t,sd,vt,et,at,seen,grth,odrr,norbs,nreal)
    return (gphs,realisable,canon)


def analyse(m,fprefix="tables/"):
    if fprefix == None:
        fname = "stdout"
    else:
        fname = fprefix+"table_"+str(m)+".txt"
    print(fname)
    with open(fname,"w") as f:
        gphs,realis,canon = generate_astral3c(m)
        print(file = f)
        print("Connected astral cyclic 3-configurations: m = "+str(m),file = f)
        print(file = f)
        nall = 0
        print("(m, j, k, t, sd?, vt?, et?, at?,  seen?, girth, nautos, norbits), nequivs", file=f)
        for g in gphs:
            res = gphs[g]
            lr = len(res)
            nall += lr 
            print(res[0],lr,file=f)
        print(file = f)
        print(file = f)
        for g in gphs:
            res = gphs[g]
            lr = len(res)
            nall += lr 
            print(res[0],lr,file=f)
            for x in res:
                print(x, file = f)
                print(file = f)
        print(file = f)
        print(file = f)

        print(file = f)
        print("Number of non-isomorphic Levi graphs ",len(gphs),file = f)
        print("Total number of admissible parameters ",nall,file = f)
        print(file = f)    
        print(file = f)
        print("Realisable parameters",file = f)
        print(file = f)
        print("(m, j, k, t, sd?, vt?, et?, at?,  seen?, girth, nautos, norbits, nintersections",file = f)
        for x in realis:
            print(file = f)
        print("Number of realizables ",len(realis),file = f)

        print(file = f)    
        print(file = f)
        print("Links to canonical parameters",file = f)
        print(file = f)
        print("(m',j',k',t') -> (m, j, k, t, sd?, vt?, et?, at?,  seen?, girth, nautos, norbits)",file = f)
        print(file = f)
        for x in canon:
            print(x,canon[x],file = f)
    
    print("Done")


In [ ]:
#os.mkdir("tables")
# reproducing tables for m = 4, 5, ..., 20
for m in range(4,21):
    analyse(m)

In [ ]:
# Test runs, manly to reproduce the figures from the paper.
plt = draw_astral_3_cfg(5,2,2,1,f=0,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(5,2,2,1,f=1,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(12,2,2,1,f=0,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(12,2,2,1,f=1,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(13,6,1,8,f=0,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(13,6,1,8,f=1,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(12,3,3,1,f=0,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(13,3,3,1,f=1,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(12,4,4,1,f=0,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(12,4,4,1,f=1,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(12,4,4,2,f=0,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(12,4,4,2,f=1,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(7,2,2,3,f=0,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(7,2,2,3,f=1,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(7,3,3,2,f=0,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(7,3,3,2,f=1,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(8,3,3,1,f=0,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(8,3,3,1,f=1,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(8,2,2,3,f=0,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(8,2,2,3,f=1,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(7,3,3,1,f=0,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(7,3,3,1,f=1,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(7,2,2,3,f=0,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(7,2,2,3,f=1,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(20,5,5,3,f=0,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(20,5,5,3,f=1,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(20,9,1,11,f=0,circ=False,r=1.5)
plt.show(axes=False)
plt = draw_astral_3_cfg(20,9,1,11,f=1,circ=False,r=1.5)
plt.show(axes=False)


